1\. zakładam bazę

In [35]:
DROP DATABASE IF EXISTS okno ;
go

CREATE DATABASE okno
/*    ON PRIMARY (
        name=okno,
        filename='/var/opt/mssql/data/okno.mdf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )
    LOG ON (
        name=okno_log,
        filename='/var/opt/mssql/data/okno.ldf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )  */
COLLATE POLISH_100_CI_AS_SC_UTF8
GO

: Msg 3702, Level 16, State 3, Line 1
Cannot drop database "okno" because it is currently in use.

: Msg 1801, Level 16, State 3, Line 4
Database 'okno' already exists. Choose a different database name.

Total execution time: 00:00:00.004

dodaję tabele   \*\*\* SŁOWNIKÓW \*\*\*

In [36]:
use okno;
GO

/* *** slowniki *** */
DROP TABLE IF EXISTS icon_name;
DROP TABLE IF EXISTS indd_master;
DROP TABLE IF EXISTS price;
DROP TABLE IF EXISTS params;
DROP TABLE IF EXISTS product_lst;
DROP TABLE IF EXISTS [name];
DROP TABLE IF EXISTS name_short;
DROP TABLE IF EXISTS name_long;
DROP TABLE IF EXISTS issue;
DROP TABLE IF EXISTS code;
DROP TABLE IF EXISTS category;
DROP TABLE IF EXISTS unit ;
DROP TABLE IF EXISTS icon;
DROP TABLE IF EXISTS icon_family;
DROP TABLE IF EXISTS unit_type;
go



CREATE TABLE category (
    id BIGINT NOT NULL IDENTITY,
    [name] varchar(127),
    exact_id BIGINT,
    category_parent BIGINT,
    exact_name varchar(127),
    PRIMARY KEY ( id )
);
go

-- slowniki kluczowe

CREATE TABLE code (
    id_code BIGINT NOT NULL IDENTITY,
    code_string varchar(32) UNIQUE,
    ean CHAR(13), -- po EAN (w tym rozwiazaniu) nie wyszukujemy, są EAN zaczynające się od 0 dlatego CHAR a nie BIGINT/DECIMAL(13)
    active BIT NOT NULL DEFAULT 1,
    exact_id BIGINT,
    category BIGINT,
    PRIMARY KEY ( id_code ),
    FOREIGN KEY ( category ) REFERENCES category( id ));
go


CREATE TABLE issue (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255) UNIQUE,
    PRIMARY KEY ( id )
);
go

-- lang zaimplementowany jako ENUM w pamięci aplikacji a nie w bazie. zmiany co kilka lat.

-- slowniki zwykle

CREATE TABLE icon_family (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255) UNIQUE,
    PRIMARY KEY ( id )
);
go


CREATE TABLE indd_master (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255) UNIQUE,
    PRIMARY KEY ( id )
);
go

CREATE TABLE unit_type (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255) UNIQUE,
    PRIMARY KEY ( id )
);
go





-- data

-- Key1

CREATE TABLE icon (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255) DEFAULT(null),

    id_unit_type BIGINT,
    id_icon_family BIGINT,

    is_tech BIT DEFAULT(0),
    is_promo BIT DEFAULT(0),

    in_paper BIT DEFAULT(0),
    in_catalog BIT DEFAULT(0),
    in_shipping BIT DEFAULT(0),
    in_rack     BIT DEFAULT(0),
    in_fish_top BIT DEFAULT(0),
    in_fish_bottom BIT DEFAULT(0),
    in_t_znamionowa BIT DEFAULT(0),
    in_t_ostrzerzen BIT DEFAULT(0),
    in_opak_jedn    BIT DEFAULT(0),
    in_opak_zbiorcze BIT DEFAULT(0),
    in_instrukcja   BIT DEFAULT(0),

    PRIMARY KEY ( id ),
    FOREIGN KEY ( id_unit_type ) REFERENCES unit_type( id ),
    FOREIGN KEY ( id_icon_family ) REFERENCES icon_family( id )
);
go  



CREATE TABLE unit (
    id BIGINT NOT NULL IDENTITY,
    [name] nvarchar(255),
    id_unit_type BIGINT NOT NULL,
    suffix nvarchar(255),
    multi DECIMAL (24,12),

    next_unit_id BIGINT NULL,
    separator nvarchar(255) NULL,

    PRIMARY KEY ( id ),
    FOREIGN KEY ( id_unit_type ) REFERENCES unit_type( id ),
);
go  



-- Key2 ( k1 x k2 )
CREATE TABLE icon_name (
    id BIGINT NOT NULL ,
    id_lang BIGINT NOT NULL,
    [name] nvarchar(255),
    PRIMARY KEY ( id, id_lang )
);
go



CREATE TABLE [name] (
    id BIGINT NOT NULL ,
    id_lang BIGINT NOT NULL,

    [name] nvarchar(255),
    name_accept INT,
    [description] nvarchar(max),
    description_accept INT,
    name_need_translate BIT DEFAULT(0),
    description_need_translate BIT DEFAULT(0),

    name_in_table nvarchar(255),
    table_header nvarchar(255),
    indd_inline_description nvarchar(255)

    PRIMARY KEY ( id, id_lang )
);
go


CREATE TABLE [name_short] (
    id BIGINT NOT NULL ,
    id_lang BIGINT NOT NULL,

    name_short nvarchar(255),
    name_accept INT,
    description_short nvarchar(max),
    description_accept BIT DEFAULT(0),

    PRIMARY KEY ( id, id_lang )
);
go

CREATE TABLE [name_long] (
    id BIGINT NOT NULL ,
    id_lang BIGINT NOT NULL,

    name_long nvarchar(255),
    name_accept INT,
    description_long nvarchar(max),
    description_accept BIT DEFAULT(0),

    PRIMARY KEY ( id, id_lang )
);
go





-- Key3 ( k1 x k2 x k3)

CREATE TABLE price (
    id BIGINT NOT NULL ,
    id_issue BIGINT NOT NULL,
    id_lang BIGINT NOT NULL,

    price DECIMAL (24,4),
    promo DECIMAL (24,4),

    PRIMARY KEY ( id, id_issue, id_lang )
);
go

CREATE TABLE product_lst (
    id BIGINT NOT NULL ,
    id_issue BIGINT NOT NULL,
    my_order INT,
    page_num INT 
    PRIMARY KEY ( id, id_issue )
);
go



DROP TABLE IF EXISTS params;
go



CREATE TABLE params (
    id_code BIGINT NOT NULL,
    id_icon BIGINT NOT NULL,

    num_value DECIMAL (24,12),
    string_value nVARCHAR(255),
    id_unit BIGINT NOT NULL
    
    PRIMARY KEY ( id_code , id_icon )
    FOREIGN KEY ( id_code ) REFERENCES dbo.code( id_code ),
    FOREIGN KEY ( id_icon ) REFERENCES dbo.icon( id ),
    FOREIGN KEY ( id_unit ) REFERENCES dbo.unit( id )

);
go  




Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.212

In [37]:

use okno;
GO

CREATE OR ALTER FUNCTION dbo.getIdByCode (
    @code_string AS nVARCHAR(32)
) RETURNS BIGINT
AS
BEGIN
    DECLARE @cID BIGINT;
    SET @cID = ( SELECT id_code FROM dbo.code WHERE code_string = @code_string );
    RETURN (@cID);
END;
go



/*
PRINT ( dbo.getIdByCode('DED8708') );

DECLARE @cID BIGINT;
SET @cID = ( dbo.getIdByCode('DED8708') );
PRINT ( @cID );
*/




CREATE OR ALTER PROCEDURE dbo.add_names
    @code_string varchar(32),
    @lang_id BIGINT,
    @i_name nvarchar(255),
    @i_description nvarchar(max)
AS
BEGIN


    DECLARE @cID BIGINT = NULL
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NOT NULL 
        BEGIN 
            IF NOT EXISTS ( SELECT id FROM dbo.[name] AS N WHERE ( N.id=@cID AND N.id_lang = @lang_id) ) 
                BEGIN 
                    -- PRINT 'INSERT';
                    INSERT INTO dbo.[name] ( id , id_lang , [name] , [description] ) VALUES ( @cID, @lang_id , @i_name , @i_description ); 
                END
            ELSE
                BEGIN 
                    -- PRINT 'UPDATE';
                    UPDATE dbo.[name]  SET  [name] = @i_name , [description] = @i_description  WHERE ( id = @cID AND id_lang = @lang_id ) ; 
                END 
        END
END
go


-------------------------------



CREATE OR ALTER PROCEDURE dbo.addprices
    @code_string varchar(32),
    @lang_id BIGINT,
    @issue_id BIGINT,    

    @i_price DECIMAL(10,2),
    @i_promo DECIMAL(10,2)
AS
BEGIN
    DECLARE @cID BIGINT = NULL;
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NULL RETURN;   
        INSERT INTO price ( id , id_lang , id_issue, price, promo ) VALUES ( @cID, @lang_id , @issue_id , @i_price, @i_promo );  
END;
go

---------------------------


CREATE OR ALTER PROCEDURE dbo.add_product_to_list
    @code_string varchar(32),
    @issue_id BIGINT,    
    @i_page TINYINT
AS
BEGIN
    DECLARE @cID BIGINT = NULL;
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NULL RETURN;   
        INSERT INTO product_lst ( id , id_issue, page_num ) VALUES ( @cID , @issue_id , @i_page );  
END;
go

 

------------------------------


CREATE OR ALTER FUNCTION dbo.getproductparams (
    @i_code BIGINT , @i_lang BIGINT  )  RETURNS TABLE
AS
RETURN 
  SELECT P.id_icon AS [icon], P.string_value AS [string], P.num_value AS [value], U.id_unit_type, U.multi,  
         I.[name] AS [PL_name], 
         I.in_paper, I.in_catalog, I.in_shipping, I.in_rack  
  FROM dbo.params AS P
    
    JOIN dbo.icon AS I ON ( P.id_icon=I.id )
    JOIN dbo.unit AS U ON ( P.id_unit=U.id )

WHERE ( P.id_code=@i_code );
GO 


------------------------------


CREATE OR ALTER FUNCTION dbo.getproduct (
    @i_code BIGINT , @i_lang BIGINT , @i_issue BIGINT )  RETURNS TABLE
AS
RETURN 
    SELECT N.[name], N.[description], P.price, P.promo  
    FROM dbo.[name] AS N 
    
    LEFT JOIN 
      dbo.price AS P ON ( P.id=@i_code AND P.ID_lang=@i_lang AND P.ID_issue=@i_issue )

    WHERE ( N.id=@i_code AND N.ID_lang=@i_lang );
GO 



Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.054

In [38]:
-- uruchamiam procedure wstawiania tresci 

USE okno;
go
/*
execute dbo.add_names  '80C051' , 1 , 'Pompa', 'do wody brudnej' ;
execute dbo.add_names  '80C051' , 2 , 'Čerpadlo', 'na špinavou vodu' ;
execute dbo.add_names  '80C051' , 3 , 'Pump', 'for dirty water' ;
execute dbo.add_names  '80C051' , 4 , 'szivattyú', 'piszkos vízhez' ;
execute dbo.add_names  '80C051' , 15 , 'Насос', 'для брудної води' ;

execute dbo.add_names  '80C052' , 1 , 'Pompa', 'do wody brudnej' ;
execute dbo.add_names  '80C052' , 2 , 'Čerpadlo', 'na špinavou vodu' ;
execute dbo.add_names  '80C052' , 3 , 'Pump', 'for dirty water' ;
execute dbo.add_names  '80C052' , 4 , 'szivattyú', 'piszkos vízhez' ;
execute dbo.add_names  '80C052' , 15 , 'Насос', 'для брудної води' ;
 
execute dbo.add_names  '0603' , 1 , 'Klucz', 'CRV - first insert' ;
execute dbo.add_names  '0603' , 1 , 'Klucz', 'CRV - second insert' ;

execute dbo.add_names 'DED8708' ,  1 ,'Kosiarka elektryczna  1600W','Moc: 1600W; prędkość obrotowa: 3600RPM; szer. koszenia: 380mm; wys. koszenia 20-70 mm (5-stopniowa regulacja); pojemność kosza: 40 L (kosz siatkowy); składana rękojeść; cicha praca' ; 
execute dbo.add_names 'DED8708' ,  2 ,'','Příkon: 1600 W, otáčky: 3600 otáček za minutu, kapacita koše: 40 l (síťový koš), 5 výšek řezu' ; 
execute dbo.add_names 'DED8708' ,  3 ,'','Power: 1600W, speed: 3600 rpm, Basket capacity: 40 L (mesh basket), 5 cutting heights' ; 
execute dbo.add_names 'DED8708' ,  4 ,'','Teljesítmény: 1600W, sebesség: 3600 fordulat/perc, kosárkapacitás: 40 L (hálós kosár), 5 vágási magasság' ; 
execute dbo.add_names 'DED8708' ,  5 ,'','Galia: 1600 W, greitis: 3600 aps/min, krepšelio talpa: 40 l (tinklinis krepšys), 5 pjovimo aukščiai' ; 
execute dbo.add_names 'DED8708' ,  6 ,'','Putere: 1600W, viteză: 3600 rpm, Capacitate co?: 40 L (co? cu plasă), 5 înăl?imi de tăiere' ; 
execute dbo.add_names 'DED8708' ,  7 ,'','Jauda: 1600 W, atrums: 3600 apgr./min, groza ietilpiba: 40 L (sieta grozs), 5 griešanas augstumi' ; 
execute dbo.add_names 'DED8708' ,  8 ,'Elektrická kosačka 1600W','Elektrická kosačka 38cm 1600W, rýchlosť otáčania: 3600 otáčok za minútu, kapacita koša: 30 l, 5 výšok kosenia (20-70 mm), šírka kosenia: 38 cm, priemer kotúča: 16 cm, 20 cm' ; 


execute dbo.add_names 'DED87308' ,  1 ,'Ostrze do kosiarki','51cm, do DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  2 ,'Nůž pro sekačku','51 cm, pro DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  3 ,'Blade for lawnmower','51cm, for DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  4 ,'Fűnyíró penge','51cm, a DED8725-51L-hez' ; 
execute dbo.add_names 'DED87308' ,  5 ,'Vejapjoves ašmenys 51cm, iki DED8725-51L','51cm, iki DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  6 ,'Lame pentru cositoare','51cm, pentru DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  7 ,'Zales plaujmašinas asmens','51cm, modelim DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  8 ,'Nôž do kosačky','51cm, do DED8725-51L' ; 
execute dbo.add_names 'DED87308' ,  16 ,'Nož kosilice 51cm za DED8725-51L','NULL' ; 
execute dbo.add_names 'DED87308' ,  17 ,'Rezilo kosilnice','51 cm, za DED8725-51L' ; 
execute dbo.add_names 'DED8823A8' ,  1 ,'Aktywna piana do mycia pojazdów 2L','Silnie skoncentrowana aktywna piana do mycia pojazdów, gruntownie usuwa zastałe, oleiste, tłuste zabrudzenia, produkt polski' ; 
execute dbo.add_names 'DED8823A8' ,  2 ,'Aktivní dvoufázová pěna 2L','Doporučován pro použití v čističích DED8819, DED8821 a DED8822' ; 
execute dbo.add_names 'DED8823A8' ,  3 ,'Active two-phase foam 2L','Recommended to use with DED8819, DED8821 and DED8822 pressure washers' ; 
execute dbo.add_names 'DED8823A8' ,  4 ,'Kétfázisú aktív hab. 2L','Ajánlott a DED8819, DED8821 és DED8822 mosó berendezésekkel használni' ; 
execute dbo.add_names 'DED8823A8' ,  5 ,'Aktyvi puta sunkiai užteršt. ireng. 2 L','Rekomenduojamas naudoti su plovimo irenginiais DED8819, DED8821 ir DED8822' ; 
execute dbo.add_names 'DED8823A8' ,  6 ,'Spumă activă cu două faze 2L','Recomandat a se utiliza cu spalatoarele DED8819, DED8821 i DED8822' ; 
execute dbo.add_names 'DED8823A8' ,  7 ,'','Augstas koncentracijas aktivas putas transportlidzeklu mazgašanai' ; 
execute dbo.add_names 'DED8823A8' ,  8 ,'Aktívna dvojfázová pena 2L','Odporúčaný na používanie s čističmi DED8819, DED8821 a DED8822' ; 
execute dbo.add_names 'DED8823A8' ,  16 ,'Dvofazna aktivna pjena 2L','NULL' ; 
execute dbo.add_names 'DED8823A8' ,  17 ,'Aktivna dvofazna pena 2L','Močno koncentrirana aktivna pena za pranje vozil Temeljito odstranjuje trdovratno, mastno in oljnato umazanijo Poljski izdelek' ; 
execute dbo.add_names 'DED8848' ,  1 ,'Pompa zanurzeniowa z zasysaniem płaskim 350W do wody czystej','Moc urządzenia: 350 W, uchwyt do przenoszenia, długość przewodu zasilającego: 10 m, przepływ: 4500 l/h, obudowa z tworzywa sztucznego odpornego na uderzenia, max.głębokość zanurzenia: 3m, max.wysokość podnoszenia: 5,5 m, max.wymiar zanieczyszczeń (ziarno): 5 mm, średnica przyłącza: 1", 5/4", klasa szczelności: IPX8' ; 
execute dbo.add_names 'DED8848' ,  2 ,'Ponorné čerpadlo 350 W s plochým sáním na čistou vodu','Výkon spotřebiče: 350 W, rukojeť pro přenášení, délka napájecího kabelu: 10 m, průtok: 4500 l/h, pouzdro z nárazuvzdorného plastu, max. hloubka ponoru: 3 m, max. výška zdvihu: 5,5 m, maximální velikost nečistot (zrna): 5 mm, průměr připojení: 1", 5/4", třída ochrany: IPX8' ; 
execute dbo.add_names 'DED8848' ,  3 ,'Low suction pump for clean water 350W','Device power: 350 W, carrying handle, power cord length: 10 m, flow rate: 4500 l/h, housing made of impact-resistant plastic, max.immersion depth: 3m, max.lifting height: 5.5m, max.size of dirt (grain): 5mm, connection diameter: 1", 5/4", protection class: IPX8' ; 
execute dbo.add_names 'DED8848' ,  4 ,'Merülő szivattyú lapos szívással 350W tiszta vízhez','A készülék teljesítménye: 350 W, hordozható fogantyú, tápkábel hossza: 10 m, áramlási sebesség: 4500 l/h, ház ütésálló műanyagból, max. merülési mélység: 3m, max. emelési magasság: 5,5 m, a szennyeződés maximális mérete (szemcseméret): 5 mm, csatlakozási átmérő: 1", 5/4", védelmi osztály: IPX8' ; 
execute dbo.add_names 'DED8848' ,  5 ,'Panardinamas siurblys su 350W plokščiu siurbimu švariam vandeniui','Prietaiso galia: 350 W, nešiojimo rankena, maitinimo laido ilgis: 10 m, srautas: 4500 l/h, korpusas pagamintas iš smugiams atsparaus plastiko, maks. panardinimo gylis: 3 m, maks. kelimo aukštis: 5,5 m, maksimalus purvo (grudeliu) dydis: 5 mm, jungties skersmuo: 1", 5/4", apsaugos klase: IPX8' ; 
execute dbo.add_names 'DED8848' ,  6 ,'Pompa submersibilă cu aspiratie plată de 350W pentru apă curată','Puterea aparatului: 350 W, mâner de transport, lungimea cablului de alimentare: 10 m, debit: 4500 l/h, carcasă din plastic rezistent la impact, adâncime maximă de imersie: 3m, înăl?imea maximă de ridicare: 5,5 m, dimensiunea maximă a pământului (boabe): 5 mm, diametrul conexiunii: 1", 5/4", clasa de protec?ie: IPX8' ; 
execute dbo.add_names 'DED8848' ,  7 ,'Plakanas sukšanas iegremdejamais suknis 350W tiram udenim','Ierices jauda: 350 W, rokturis parnesašanai, barošanas kabela garums: 10 m, plusmas atrums: 4500 l/h, korpuss izgatavots no triecienizturigas plastmasas, maks. iegremdešanas dzilums: 3 m, maksimalais pacelšanas augstums: 5,5 m, maksimalais netirumu (graudu) izmers: 5 mm, savienojuma diametrs: 1", 5/4", aizsardzibas klase: IPX8' ; 
execute dbo.add_names 'DED8848' ,  8 ,'Ponorné čerpadlo s plochým saním 350 W na čistú vodu','Výkon spotrebiča: 350 W, rukoväť na prenášanie, dĺžka napájacieho kábla: 10 m, prietok: 4500 l/h, puzdro z nárazuvzdorného plastu, max. hĺbka ponoru: 3 m, max. výška zdvihu: 5,5 m, max. veľkosť nečistôt (zrna): 5 mm, priemer pripojenia: 1", 5/4", trieda ochrany: IPX8' ; 
execute dbo.add_names 'DED8848' ,  16 ,'Potopna pumpa s ravnim usisom od 350W za čistu vodu','NULL' ; 
execute dbo.add_names 'DED8848' ,  17 ,'Potopna črpalka z ravnim sesanjem 350 W za čisto vodo','Napajanje naprave: 350 W, ročaj za prenašanje, dolžina napajalnega kabla: 10 m, pretok: 4500 l/h, ohišje iz plastike, odporne na udarce, največja globina potopitve: 3 m, največja višina dviga: 5,5 m, največja velikost umazanije (zrna): 5 mm, premer priključka: 1", 5/4", zaščitni razred: IPX8' ; 

go


---------------


Declare @Issue BIGINT = 1
Declare @Lang BIGINT = 1 -- 1=PL, 2=CZ, 3=EN, 15=UA

EXEC dbo.addprices '80C051' , @Lang , @Issue , 3.21 , 2.99 ;
EXEC dbo.addprices '80C052' , @Lang , @Issue , 4.21 , 3.99 ;
EXEC dbo.addprices '0602' , @Lang , @Issue , 5.21 , 4.99 ;
EXEC dbo.addprices '0603' , @Lang , @Issue , 6.21 , 4.99 ;
EXEC dbo.addprices 'DED780911' , @Lang , @Issue , 7.21 , 5.99 ;
EXEC dbo.addprices 'DED780912' , @Lang , @Issue , 8.21 , 6.99 ;
EXEC dbo.addprices 'DED780920' , @Lang , @Issue , 11.21 , 8.99 ;

EXEC dbo.addprices '80C051' , 2 , @Issue , 23.21 , 22.99 ;
EXEC dbo.addprices '80C052' , 2 , @Issue , 24.21 , 23.99 ;
EXEC dbo.addprices '80C051' , 3 , @Issue , 33.21 , 32.99 ;
EXEC dbo.addprices '80C052' , 3 , @Issue , 34.21 , 33.99 ;
EXEC dbo.addprices '80C051' , 4 , @Issue , 34.21 , 42.99 ;
EXEC dbo.addprices '80C052' , 4 , @Issue , 44.21 , 43.99 ;
EXEC dbo.addprices '80C051' , 15 , @Issue , 153.21 , 152.99 ;
EXEC dbo.addprices '80C052' , 15 , @Issue , 154.21 , 153.99 ;

 
---------------------

EXEC dbo.add_product_to_list '80C051'  , @Issue  , 2 ;
EXEC dbo.add_product_to_list '80C052'  , @Issue  , 2 ;
EXEC dbo.add_product_to_list '0602'    , @Issue  , 2 ;
EXEC dbo.add_product_to_list '0603'    , @Issue  , 2 ;
EXEC dbo.add_product_to_list 'DED780911', @Issue , 2 ;
EXEC dbo.add_product_to_list 'DED780912', @Issue , 2 ;
EXEC dbo.add_product_to_list 'DED780920', @Issue , 2 ;
go

-----------------------


INSERT INTO params (id_code, id_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C051' ), 1, 120, NULL , 9 );
INSERT INTO params (id_code, id_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C052' ), 1, 90, NULL , 9 );
INSERT INTO params (id_code, id_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C051' ), 3, NULL, NULL , 1);
INSERT INTO params (id_code, id_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C052' ), 3, NULL, NULL , 1 );

----------------
*/

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007

In [39]:
USE okno;
go


-- WYPEŁNIAM SŁOWNIKI     
/*  Słownik nieużywany - dane zapisane w pamieci aplikacji w zmiennej ENUM  
INSERT INTO lang VALUES (1,'PL','zł',2,'nr. kat.|%|cena|promocja');
INSERT INTO lang VALUES (2,'CZ','Kč',1,'kat. č.|popis|cena|akční cena');
INSERT INTO lang VALUES (3,'EN','€',2, 'cat. no.|description|price|promo price');
INSERT INTO lang VALUES (4,'HU','Ft',0,'cikkszám|leírás|ár|akciós ár');
-- ****
INSERT INTO lang VALUES (15,'UA','€',2,'Код|опис|ціна|акційна ціна');
*/

-----------------
/*
INSERT INTO issue ( [name] ) VALUES ( 'ONE 23.01');


-----------------

INSERT INTO category(exact_id, [name]) VALUES( 10000 , 'Kosiarki'); 
INSERT INTO category(exact_id, [name]) VALUES( 10001 , '- Kosiarki spalinowe'); 
INSERT INTO category(exact_id, [name]) VALUES( 10002 , '- Kosiarki elektryczne'); 
INSERT INTO category(exact_id, [name]) VALUES( 20000 , 'Ślimacznice'); 

-----------------

DECLARE @DED8708 AS BIGINT;
INSERT INTO code ( code_string, ean ) VALUES ( 'DED8708' ,'5902628870803');  --3008
SELECT @DED8708 = SCOPE_IDENTITY(); 
PRINT(@DED8708);
INSERT INTO code ( code_string, ean ) VALUES ( 'DED87308' , '5902628873088'); --3036

INSERT INTO code ( code_string, ean ) VALUES ( 'DED8823A8', '5902628882417');  --3082
INSERT INTO code ( code_string, ean ) VALUES ( 'DED8848', '5902628884800');  --4373
INSERT INTO code ( code_string  ) VALUES ( '80C051' );   
INSERT INTO code ( code_string  ) VALUES ( '80C052' );   
INSERT INTO code ( code_string  ) VALUES ( '0603' );   


UPDATE code SET category=( SELECT MAX (id) FROM category WHERE [name]='- Kosiarki spalinowe' ) WHERE code_string='DED8708';  
UPDATE code SET category=( SELECT MAX (id) FROM category WHERE [name]='- Kosiarki spalinowe' ) WHERE code_string='DED87308';  
UPDATE code SET category=( SELECT MAX (id) FROM category WHERE [name]='- Kosiarki spalinowe' ) WHERE code_string='DED8823A8';  



INSERT INTO icon_family ( [name]  ) VALUES ('maszyny z silnikami spalinowymi');
DECLARE @fam AS BIGINT;
SET @fam = ( SELECT MAX (id) FROM icon_family WHERE name='maszyny z silnikami spalinowymi');



INSERT INTO unit_type ([name]) VALUES ( '[NONE]' ); -- miara dlugosci
DECLARE @none AS BIGINT;
SET @none = ( SELECT MAX (id) FROM unit_type WHERE name='[NONE]');

INSERT INTO unit_type ([name]) VALUES ( 'dlugosc' ); -- miara dlugosci
DECLARE @dlugosc AS BIGINT;
SET @dlugosc = ( SELECT MAX (id) FROM unit_type WHERE name='dlugosc');

INSERT INTO unit_type ([name]) VALUES ( 'moc' ); -- miara mocy
DECLARE @moc AS BIGINT;
SET @moc = ( SELECT MAX (id) FROM unit_type WHERE name='moc');


INSERT INTO unit_type ([name]) VALUES ( 'silnik bezszczotkowy' ); -- rodzaj silnika
DECLARE @bezsz AS BIGINT;
SET @bezsz = ( SELECT MAX (id) FROM unit_type WHERE name='silnik bezszczotkowy');


DECLARE @icoa AS BIGINT; DECLARE @icob AS BIGINT; DECLARE @icoc AS BIGINT; 
INSERT INTO icon ( id_unit_type , id_icon_family, [name], is_tech, in_rack, in_catalog  )   VALUES  (   @moc , @fam ,'Moc silnika elektycznego',1 ,1, 1 ); --is_tect, is_promo okresla dział odpowiedzialny za parametr
SELECT @icoa = SCOPE_IDENTITY();
INSERT INTO icon ( id_unit_type , id_icon_family, [name], is_tech, in_rack, in_catalog  )   VALUES  (   @moc , @fam, 'Moc silnika spalinowego' ,1 ,1, 1 ); -- in_rack, in_catal  określa w jakich publkacjach lub w jakiego rodzaju dokumentach pojawi się ikona
SELECT @icob = SCOPE_IDENTITY();
INSERT INTO icon ( id_unit_type,  id_icon_family, [name], is_promo, in_catalog  )           VALUES  (  @bezsz, @fam, 'Silnik bezszczotkowy'  ,1, 1);
SELECT @icoc = SCOPE_IDENTITY(); 
-----------------

INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoa,1,'Moc silnika elektycznego');
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icob,1,'Moc silnika spalinowego' );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoc,1,'Silnik bezszczotkowy'   ); 
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoa,2,'Výkon elektromotoru');
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icob,2,'Výkon spalovacího motoru' );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoc,2,'Bezkartáčový motor'    );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoa,3,'Power of the electric engine');
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icob,3,'Internal combustion engine power' );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoc,3,'Brushless motor'    );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoa,15,'Потужність електродвигуна');
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icob,15,'Потужність двигуна внутрішнього згоряння' );
INSERT INTO icon_name ( id, id_lang, [name]  ) VALUES (@icoc,15,'Безщітковий двигун'    );



-----------------

INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@none,'[ NONE ]','[ *** NONE *** ]' , 1);
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@dlugosc,'km','kilometr' , 1000);
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@dlugosc,'m','metr'      , 1);
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@dlugosc,'dm','decyment' , 0.1);
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@dlugosc,'cm','centymetr', 0.01);
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@dlugosc,'mm','milimetr' , 0.001);

INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@moc,'MW','megawat' , 1000000);   DECLARE @kw AS BIGINT;
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@moc,'kW','kilowat' , 1000);      SELECT @kw = SCOPE_IDENTITY();
INSERT INTO unit ( id_unit_type, [name], suffix, multi  ) VALUES (@moc,'W','wat' , 1);



INSERT INTO params ( id_code, id_icon, num_value, id_unit ) VALUES (  @DED8708 ,  @icoa , 1.2 , @kw ); --moc asilnika el. 


*/

go





Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007

tworzę tabelę indd\_name wraz z procedurą

In [46]:

USE okno;
GO


DECLARE @PL BIGINT = 1;
DECLARE @CZ BIGINT = 2;
DECLARE @EN BIGINT = 3;
DECLARE @UA BIGINT = 15;


-- nowe wydanie
                                                        DECLARE @STYCZEN_2023 BIGINT ;
INSERT INTO issue ( [name] ) VALUES ( 'ONE 23.01');     SELECT @STYCZEN_2023 = SCOPE_IDENTITY(); 


-- dodaję kody -- powinien to robić automat - ładować z innego syustemu liste dostęnych kodów
                                                                                 
INSERT INTO code ( code_string, ean ) VALUES ( 'DED8708' ,'5902628870803');    

-- nazwy
execute dbo.add_names 'DED8708' ,  1 ,'Kosiarka elektryczna  1600W','Moc: 1600W; prędkość obrotowa: 3600RPM; szer. koszenia: 380mm; wys. koszenia 20-70 mm (5-stopniowa regulacja); pojemność kosza: 40 L (kosz siatkowy); składana rękojeść; cicha praca' ; 
execute dbo.add_names 'DED8708' ,  2 ,'','Příkon: 1600 W, otáčky: 3600 otáček za minutu, kapacita koše: 40 l (síťový koš), 5 výšek řezu' ; 
execute dbo.add_names 'DED8708' ,  3 ,'','Power: 1600W, speed: 3600 rpm, Basket capacity: 40 L (mesh basket), 5 cutting heights' ; 
execute dbo.add_names 'DED8708' ,  4 ,'','Teljesítmény: 1600W, sebesség: 3600 fordulat/perc, kosárkapacitás: 40 L (hálós kosár), 5 vágási magasság' ; 
execute dbo.add_names 'DED8708' ,  5 ,'','Galia: 1600 W, greitis: 3600 aps/min, krepšelio talpa: 40 l (tinklinis krepšys), 5 pjovimo aukščiai' ; 
execute dbo.add_names 'DED8708' ,  6 ,'','Putere: 1600W, viteză: 3600 rpm, Capacitate co?: 40 L (co? cu plasă), 5 înăl?imi de tăiere' ; 
execute dbo.add_names 'DED8708' ,  7 ,'','Jauda: 1600 W, atrums: 3600 apgr./min, groza ietilpiba: 40 L (sieta grozs), 5 griešanas augstumi' ; 
execute dbo.add_names 'DED8708' ,  8 ,'Elektrická kosačka 1600W','Elektrická kosačka 38cm 1600W, rýchlosť otáčania: 3600 otáčok za minútu, kapacita koša: 30 l, 5 výšok kosenia (20-70 mm), šírka kosenia: 38 cm, priemer kotúča: 16 cm, 20 cm' ; 

--

INSERT INTO category(exact_id, [name]) VALUES( 10000 , 'Kosiarki'); 
INSERT INTO category(exact_id, [name]) VALUES( 10001 , '- Kosiarki spalinowe'); 
INSERT INTO category(exact_id, [name]) VALUES( 10002 , '- Kosiarki elektryczne'); 

--


EXEC dbo.addprices 'DED8708' , 1 , @STYCZEN_2023 , 2.21 , 1.99 ;
EXEC dbo.addprices 'DED8708' , 2 , @STYCZEN_2023 , 3.21 , 12.99 ;
EXEC dbo.addprices 'DED8708' , 3 , @STYCZEN_2023 , 4.21 , 3.99 ;
EXEC dbo.addprices 'DED8708' , 15 , @STYCZEN_2023 , 6.21 , 5.99 ;


-- wprowadzam typy jednostek
INSERT INTO unit_type ([name]) VALUES ( '[NONE]' ); -- miara dlugosci
    DECLARE @none AS BIGINT; SET @none = ( SELECT MAX (id) FROM unit_type WHERE name='[NONE]');

INSERT INTO unit_type ([name]) VALUES ( 'dlugosc' ); -- miara dlugosci
    DECLARE @dlugosc AS BIGINT; SET @dlugosc = ( SELECT MAX (id) FROM unit_type WHERE name='dlugosc');

INSERT INTO unit_type ([name]) VALUES ( 'moc' ); -- miara mocy
    DECLARE @moc AS BIGINT; SET @moc = ( SELECT MAX (id) FROM unit_type WHERE name='moc');

-- wprowadzam jednostki
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@none,'[ NONE ]','[ *** NONE *** ]' , 1);
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@dlugosc,'km','kilometr' , 1000);
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@dlugosc,'m','metr'      , 1);
INSERT INTO unit ( id_unit_type, suffix, [name], multi  )  VALUES (@dlugosc,'dm','decyment' , 0.1);    DECLARE @cm AS BIGINT;
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@dlugosc,'cm','centymetr', 0.01);     SELECT @cm = SCOPE_IDENTITY();
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@dlugosc,'mm','milimetr' , 0.001);

INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@moc,'MW','megawat' , 1000000);   DECLARE @kw AS BIGINT;
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@moc,'kW','kilowat' , 1000);      SELECT @kw = SCOPE_IDENTITY();
INSERT INTO unit ( id_unit_type, suffix, [name], multi  ) VALUES (@moc,'W','wat' , 1);

-- wprowadzam ikony, określam ich typy_jednostek
        DECLARE @ico_moc AS BIGINT; 
INSERT INTO icon ( id_unit_type , [name] )           VALUES  (  @moc, 'Silnik bezszczotkowy');
        SELECT @ico_moc = SCOPE_IDENTITY();


        DECLARE @ico_dlugosc AS BIGINT; 
INSERT INTO icon ( id_unit_type , [name] )   VALUES  (  @dlugosc, 'długość');
        SELECT @ico_dlugosc = SCOPE_IDENTITY();

INSERT INTO params ( id_code, id_icon, num_value, id_unit ) VALUES (  1 ,  @ico_moc , 1.2 , @kw ); --moc asilnika el.         
INSERT INTO params ( id_code, id_icon, num_value, id_unit ) VALUES (  1 ,  @ico_dlugosc , 33 , @cm ); --moc asilnika el. 

-- pobieram nazwy i opisy w 3 jezykach
SELECT * FROM dbo.getproduct   ( 1,1,@STYCZEN_2023 );
SELECT * FROM dbo.getproduct   ( 1,2,@STYCZEN_2023 );
SELECT * FROM dbo.getproduct   ( 1,3,@STYCZEN_2023 );

-- pobieram opisy w 3 językach

SELECT * FROM dbo.getproductparams( dbo.getIdByCode( 'DED8708' ) , @PL  );
SELECT * FROM dbo.getproductparams( dbo.getIdByCode( 'DED8708' ) , @EN  );
SELECT * FROM dbo.getproductparams( dbo.getIdByCode( 'DED8708' ) , @CZ  );




Commands completed successfully.

The statement has been terminated.

The statement has been terminated.

(1 row affected)

(1 row affected)

...

: Msg 2627, Level 14, State 1, Line 14
Violation of UNIQUE KEY constraint 'UQ__issue__72E12F1B47D2C3E2'. Cannot insert duplicate key in object 'dbo.issue'. The duplicate key value is (ONE 23.01).

: Msg 2627, Level 14, State 1, Line 19
Violation of UNIQUE KEY constraint 'UQ__code__7B2FA2B62935D634'. Cannot insert duplicate key in object 'dbo.code'. The duplicate key value is (DED8708).

: Msg 2627, Level 14, State 1, Line 47
Violation of UNIQUE KEY constraint 'UQ__unit_typ__72E12F1B9C1F96DE'. Cannot insert duplicate key in object 'dbo.unit_type'. The duplicate key value is ([NONE]).

: Msg 2627, Level 14, State 1, Line 50
Violation of UNIQUE KEY constraint 'UQ__unit_typ__72E12F1B9C1F96DE'. Cannot insert duplicate key in object 'dbo.unit_type'. The duplicate key value is (dlugosc).

: Msg 2627, Level 14, State 1, Line 53
Violation of UNIQUE KEY constraint 'UQ__unit_typ__72E12F1B9C1F96DE'. Cannot insert duplicate key in object 'dbo.unit_type'. The duplicate key value is (moc).